In [ ]:
%matplotlib inline

In [ ]:
from datetime import timedelta

import geopandas as gpd
import numpy as np
import pandas as pd
from eolearn.core import (
    EOExecutor,
    EOPatch,
    EOTask,
    EOWorkflow,
    FeatureType,
    OverwritePermission,
    SaveTask,
)
from eolearn.io import SentinelHubInputTask
from fs_s3fs import S3FS
from matplotlib import pyplot as plt
from sentinelhub import DataCollection, SHConfig, UtmZoneSplitter

# Config

In [ ]:
config = SHConfig()
config.sh_client_id = ''
config.sh_client_secret = ''
config.instance_id = ''
config.aws_access_key_id = ''
config.aws_secret_access_key = ''

In [ ]:
test_area_cyprus = gpd.read_file('../input/TestAreaCyprus.geojson')
test_area_lithuania = gpd.read_file('../input/TestAreaLithuania.geojson')

In [ ]:
filesystem = S3FS(bucket_name='',
                  aws_access_key_id=config.aws_access_key_id,
                  aws_secret_access_key=config.aws_secret_access_key)

# Load metadata

In [ ]:
metadata_ms4 = pd.read_parquet(filesystem.openbin('metadata/deimos_ms4_metadata.pq'))
metadata_pan = pd.read_parquet(filesystem.openbin('metadata/deimos_pan_metadata.pq'))

# Split into small bboxes

In [ ]:
bbox_splitter = UtmZoneSplitter(test_area_lithuania.geometry.to_list(
) + test_area_cyprus.geometry.to_list(), crs=test_area_lithuania.crs, bbox_size=2000)

In [ ]:
bboxes = bbox_splitter.get_bbox_list(buffer=0.2)

# Download data from SH service

Downloads data:

1. Sentinel-2 data in interval 2020-04-01 - 2020-10-01
2. Deimos MS4 bands
3. Deimos PAN band
4. Deimos pansharpened

Downloads everything as digital numbers. The Deimos data is saved to one eopatch, S-2 data to another. This is due to the fact that the acquisition timestamps are different for these two data sources. Metadata is also added to S-2 bands that can be used for normalizing.

In [ ]:
evalscript_pansharp = '''
//VERSION=3

function setup() {
    return {
        input: [{
            bands: ["B01", "B02", "B03", "B04", "PAN", "dataMask"],
            units: ["DN", "DN", "DN", "DN", "DN", "DN"]
        }],
        output: [
            { id:"bands", bands:4, sampleType: SampleType.UINT16 }, 
            { id:"bool_mask", bands:1, sampleType: SampleType.UINT8 },
        ]
    }
}

function updateOutputMetadata(scenes, inputMetadata, outputMetadata) {
    outputMetadata.userData = { "norm_factor":  inputMetadata.normalizationFactor }
}

function evaluatePixel(sample) {
  let sudoPanW = (sample.B01 + sample.B02 + sample.B03 + sample.B04) / 4
  let ratioW = sample.PAN / sudoPanW
  let red = sample.B02 * ratioW
  let green = sample.B03 * ratioW
  let blue = sample.B04 * ratioW
  let nir = sample.B01 * ratioW 
  return {bands: [blue, green, red, nir], bool_mask: [sample.dataMask]};
}
'''

In [ ]:
# Download Sentinel-2 data

get_s2_data = SentinelHubInputTask(
    bands_feature=(FeatureType.DATA, 'BANDS'),
    bands=['B02', 'B03', 'B04', 'B08'],
    resolution=10,
    maxcc=0.5,
    time_difference=timedelta(minutes=120),
    data_collection=DataCollection.SENTINEL2_L1C,
    additional_data=[(FeatureType.MASK, 'dataMask', 'IS_DATA'),
                     (FeatureType.MASK, 'CLM'),
                     (FeatureType.DATA, 'CLP')],
    max_threads=5,
    config=config,
    bands_dtype=np.uint16
)

# Download pansharpened Deimos.
get_deimos_data_pansharpened = SentinelHubInputTask(
    bands_feature=(FeatureType.DATA, 'BANDS-DEIMOS'),
    bands=['B01', 'B02', 'B03', 'B04'],  # B, G, R, NIR
    resolution=2.5,
    time_difference=timedelta(minutes=120),
    data_collection=DataCollection.define_byoc(''), # INPUT BYOC COLLECTION ID HERE
    additional_data=[(FeatureType.MASK, 'dataMask', 'IS_DATA')],
    max_threads=5,
    evalscript=evalscript_pansharp,
    config=config,
    bands_dtype=np.uint16,
    aux_request_args=dict(processing=dict(upsampling='BICUBIC',
                                          downsampling='BICUBIC'))
)


save_s2 = SaveTask('', # INPUT WHERE TO SAVE S-2 PATCHES
                   config=config,
                   overwrite_permission=OverwritePermission.OVERWRITE_PATCH)
save_dm = SaveTask('', # INPUT WHERE TO SAVE Deimos PATCHES
                   config=config,
                   overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)

In [ ]:
class AddMetaData(EOTask):
    """ Adds metadata to Deimos EOPatch. Uses dataframe with metadata that was parsed from tile .dim files. """

    def __init__(self, metadata_ms4, metadata_pan):
        self.metadata_ms4 = metadata_ms4
        self.metadata_pan = metadata_pan

    def _create_meta_data_dict(self, eop):
        meta_info_dict = {}
        for ts in eop.timestamp:
            metadata_pan_ts = self.metadata_pan[self.metadata_pan.START_TIME ==
                                                ts.strftime('%Y-%m-%dT%H:%M:%S')].reset_index()
            metadata_ms4_ts = self.metadata_ms4[self.metadata_ms4.START_TIME ==
                                                ts.strftime('%Y-%m-%dT%H:%M:%S')].reset_index()

            bands_stats_ms4 = metadata_ms4_ts[['STX_STDV_1', 'STX_MEAN_1', 'STX_MIN_1',
                                               'STX_MAX_1', 'STX_STDV_2', 'STX_MEAN_2',
                                               'STX_MIN_2', 'STX_MAX_2', 'STX_STDV_3',
                                               'STX_MEAN_3', 'STX_MIN_3', 'STX_MAX_3',
                                               'STX_STDV_4', 'STX_MEAN_4', 'STX_MIN_4', 'STX_MAX_4']].to_dict(orient='index')[0]

            bands_physical_info_ms4 = metadata_ms4_ts[['PHYSICAL_GAIN_1', 'PHYSICAL_BIAS_1', 'PHYSICAL_UNIT_1',
                                                       'ESUN_1', 'PHYSICAL_GAIN_2', 'PHYSICAL_BIAS_2',
                                                       'PHYSICAL_UNIT_2', 'ESUN_2', 'PHYSICAL_GAIN_3',
                                                       'PHYSICAL_BIAS_3', 'PHYSICAL_UNIT_3', 'ESUN_3',
                                                       'PHYSICAL_GAIN_4', 'PHYSICAL_BIAS_4', 'PHYSICAL_UNIT_4',
                                                       'ESUN_4']].to_dict(orient='index')[0]

            bands_stats_pan = metadata_pan_ts[['STX_STDV_4', 'STX_MEAN_4',
                                               'STX_MIN_4', 'STX_MAX_4']].to_dict(orient='index')[0]

            bands_physical_info_pan = metadata_pan_ts[['PHYSICAL_GAIN_4', 'PHYSICAL_BIAS_4',
                                                       'PHYSICAL_UNIT_4',
                                                       'ESUN_4']].to_dict(orient='index')[0]

            meta_info_dict[ts] = {'MS4': {'BAND_STATS': bands_stats_ms4, 'PHYSICAL_INFO': bands_physical_info_ms4},
                                  'PAN': {'BAND_STATS': bands_stats_pan, 'PHYSICAL_INFO': bands_physical_info_pan}}
        return meta_info_dict

    def execute(self, eop):
        meta_data_dict = self._create_meta_data_dict(eop)
        eop.meta_info['metadata'] = meta_data_dict
        return eop

In [ ]:
add_metadata = AddMetaData(metadata_ms4, metadata_pan)

In [ ]:
# Execute the workflow
time_interval = ['2020-04-01', '2020-10-31']  # time interval for the SH request

# define additional parameters of the workflow
execution_args = []
for idx, bbox in enumerate(bboxes):
    execution_args.append({
        get_deimos_data_pansharpened: {'bbox': bbox, 'time_interval': time_interval},
        get_s2_data: {'bbox': bbox, 'time_interval': time_interval},
        save_s2: {'eopatch_folder': f'eopatch-{idx:04d}'},
        save_dm: {'eopatch_folder': f'eopatch-{idx:04d}'}
    })

In [ ]:
workflow = EOWorkflow([
    (get_s2_data, [], 'Get S2 data'),
    (get_deimos_data_pansharpened, [], 'Get Deimos pansharpened'),
    (add_metadata, [get_deimos_data_pansharpened], 'Add metadata to DEIMOS'),
    (save_s2, [get_s2_data], 'save S2 data'),
    (save_dm, [add_metadata], 'save deimos data')
])

In [ ]:
len(execution_args)

In [ ]:
executor = EOExecutor(workflow, execution_args, save_logs=True)
results = executor.run(workers=10, multiprocess=False)
executor.make_report()

In [ ]:
eop_d = EOPatch.load('', # SAMPLE DEIMOS PATCH PATH
                     filesystem=filesystem)
eop_s2 = EOPatch.load('', # SAMPLE S-2 PATCH PATH
                      filesystem=filesystem)

In [ ]:
eop_d

In [ ]:
fig, ax = plt.subplots(2, figsize=(15, 15))
ax[0].imshow(eop_d.data['BANDS-DEIMOS'][5][..., [2, 1, 0]] / 10000 * 1.5)
ax[1].imshow(eop_s2.data['BANDS'][-7][..., [2, 1, 0]] / 10000 * 3.5)